In [ ]:
import sys
from pathlib import Path

root_dir = Path.cwd().parent
sys.path.append(str(root_dir))

import numpy as np  # noqa: E402
import dpdata  # noqa: E402
from IPython.display import Image, display  # noqa: E402

from ai2_kit.feat.spectrum import md_spectra  # noqa: E402

output_dir_name = "show_output"
current_path = Path.cwd()
output_dir = current_path / output_dir_name

if not output_dir.exists():
    output_dir.mkdir(parents=True, exist_ok=True)

sample_dir = root_dir / "tests" / "data-sample" / "md_spectra_sample"

c:\Users\nagoriyuki\miniconda3\envs\ai2-kit-rin\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'WindowsPath' object has no attribute 'exist'

从分子动力学轨迹数据中提取水分子和羟基的氧原子坐标，并结合 Wannier 中心数据计算水和羟基的原子偶极矩，结果保存为 NumPy 文件，便于后续光谱等分析。

In [ ]:
a0 = 0.52917721067
traj = dpdata.System(sample_dir / "traj", fmt="deepmd/npy")
wannier: np.ndarray = np.load(sample_dir / "wannier_dipole.npy")
wannier = wannier.reshape(traj.get_nframes(), -1, 3)
name_h2o = "h2o.npy"
name_adw = "atomic_dipole_wan.npy"

md_spectra.compute_atomic_dipole_h2o(
    traj=traj,
    wannier=wannier,
    type_O=1,
    type_H=2,
    r_bond=1.3,
    a0=a0,
    save_datas=[output_dir / name_h2o, output_dir / name_adw],
)

处理水分子（H2O）和羟基（OH）的极化数据。通过 DPData 库读取原子轨迹数据，对坐标应用周期性边界条件，然后识别体系中的 H2O 和 OH 基团。脚本从 Wannier 极化数据中提取与这些基团相关的信息，并分别保存为 H2O 和 OH 的极化数据文件，以便后续分析或模拟使用。

In [ ]:
traj = dpdata.System(sample_dir / "traj", fmt="deepmd/npy")
polar: np.ndarray = np.load(sample_dir / "wannier_polar.npy")
polar = -polar.reshape(polar.shape[0], -1, 3, 3)
name = "atomic_polar_wan.npy"

md_spectra.extract_atomic_polar_from_traj_h2o(
    traj=traj,
    polar=polar,
    type_O=1,
    type_H=2,
    r_bond=1.3,
    save_data=output_dir / name,
)

计算和绘制水分子的红外 (IR) 光谱。从文件中加载水分子的坐标、晶胞和原子偶极矩数据，应用空间权重函数区分不同区域的分子，然后计算分子内和分子间偶极矩的自相关函数。通过傅里叶变换将这些相关函数转换为频率域数据，最终生成并保存红外光谱图，展示波数与强度的关系。

In [ ]:
dt = 0.0005
window = 2000

h2o = np.load(sample_dir / "traj/set.000/h2o.npy")
cells = np.load(sample_dir / "traj/set.000/box.npy").reshape(h2o.shape[0], 3, 3)
atomic_dipole = np.load(sample_dir / "traj/set.000/atomic_dipole_wan_h2o.npy").reshape(h2o.shape[0], -1, 3)
name = "ir_spectrum.npy"

md_spectra.compute_bulk_ir_h2o(
    h2o=h2o,
    cells=cells,
    atomic_dipole=atomic_dipole,
    dt=dt,
    window=window,
    z0=21.5,
    zc=5.0,
    zw=0.5,
    rc=6.0,
    width=240,
    temperature=330.0,
    M=20000,
    filter_type="lorenz",
    save_plot=output_dir / "bulk_ir.png",
    save_data=output_dir / name,
)

display(Image(filename=output_dir / 'bulk_ir.png'))

计算并绘制水分子的拉曼光谱。加载水分子坐标、晶胞和原子极化率数据，应用空间权重函数区分不同区域的分子，计算分子内和分子间的各向同性与各向异性极化率自相关函数。通过傅里叶变换将这些相关函数转换为频域信号，生成并保存三种拉曼光谱图：各向同性光谱 (2800-4000 cm⁻¹)、各向异性光谱 (2800-4000 cm⁻¹) 以及低频各向异性光谱 (0-2500 cm⁻¹)。

In [ ]:
dt = 0.0005
window = 2000

h2o = np.load(sample_dir / "traj/set.000/h2o.npy")
cells = np.load(sample_dir / "traj/set.000/box.npy").reshape(h2o.shape[0], 3, 3)
atomic_polar = np.load(sample_dir / "traj/set.000/atomic_polar_wan_h2o.npy").reshape(h2o.shape[0], -1, 3, 3)
name = "br.npy"

md_spectra.compute_bulk_raman_h2o(
    h2o=h2o,
    cells=cells,
    atomic_polar=atomic_polar,
    dt=dt,
    window=window,
    z0=21.5,
    zc=5.0,
    zw=0.5,
    rc=6.0,
    width=240,
    temperature=330.0,
    M=20000,
    filter_type="lorenz",
    save_plots=[
        output_dir / "bulk_raman_iso.png",
        output_dir / "bulk_raman_aniso.png",
        output_dir / "bulk_raman_aniso_low.png",
    ],
    save_data=output_dir / name,
)

display(Image(filename=output_dir / "bulk_raman_iso.png"))
display(Image(filename=output_dir / "bulk_raman_aniso.png"))
display(Image(filename=output_dir / "bulk_raman_aniso_low.png"))

计算和绘制水分子（H₂O）与羟基（OH）的红外光谱。加载两者的坐标、晶胞及原子偶极矩数据并合并，通过空间权重函数处理特定区域的分子，计算分子内和分子间偶极矩自相关函数，再经傅里叶变换得到频域数据，分别生成 H₂O 和 OH 的红外光谱图，展示不同基团在 0-4000 cm⁻¹ 波数范围内的红外吸收强度。

In [ ]:
dt = 0.0005
window = 50000

h2o = np.load(sample_dir / "h2o.npy")
atomic_dipole = np.load(sample_dir / "atomic_dipole_wan.npy")
atomic_dipole = atomic_dipole.reshape(atomic_dipole.shape[0], -1, 3)
name = "ir_sp.dat"

md_spectra.compute_surface_ir_spectra_h2o(
    h2o=h2o,
    atomic_dipole=atomic_dipole,
    dt=dt,
    window=window,
    z1_min=16.0,
    z1_max=17.4,
    z2_min=20.0,
    z2_max=25.0,
    z3_min=27.6,
    z3_max=29.0,
    z_total_min=16.0,
    z_total_max=29.0,
    z_bin=0.4,
    width=25,
    temperature=330.0,
    save_plot=output_dir / "ir_sp.png",
    save_data=output_dir / name,
)

display(Image(output_dir / "ir_sp.png"))

计算和绘制水分子（H₂O）与羟基（OH）体系的拉曼光谱。加载两者的坐标、晶胞及原子极化率数据并合并，通过空间权重函数处理特定区域的分子，计算分子内和分子间的各向同性与各向异性极化率自相关函数，经傅里叶变换得到频域信号后，分别生成 H₂O 的各向同性光谱（2800-4000 cm⁻¹）、各向异性光谱（2800-4000 cm⁻¹）及低频各向异性光谱（0-2500 cm⁻¹）并保存为图像文件。

In [ ]:
dt = 0.0005
window = 2000

h2o = np.load(sample_dir / "traj/set.000/h2o.npy")
oh = np.load(sample_dir / "traj/set.000/oh.npy")
cells = np.load(sample_dir / "traj/set.000/box.npy").reshape(h2o.shape[0], 3, 3)
atomic_polar_h2o = np.load(sample_dir / "traj/set.000/atomic_polar_wan_h2o.npy").reshape(h2o.shape[0], -1, 3, 3)
atomic_polar_oh = np.load(sample_dir / "traj/set.000/atomic_polar_wan_oh.npy").reshape(h2o.shape[0], -1, 3, 3)
name = "sr.npy"

md_spectra.compute_surface_raman_h2o(
    h2o=h2o,
    oh=oh,
    cells=cells,
    atomic_polar_h2o=atomic_polar_h2o,
    atomic_polar_oh=atomic_polar_oh,
    dt=dt,
    window=window,
    z0=21.5,
    zc=10.0,
    zw=0.5,
    rc=6.0,
    width=240,
    temperature=330.0,
    M=20000,
    filter_type="lorenz",
    save_plots=[
        output_dir / "sur_raman_iso.png",
        output_dir / "sur_raman_aniso.png",
        output_dir / "sur_raman_aniso_low.png",
    ],
    save_data=output_dir / name,
)

display(Image(output_dir / "sur_raman_iso.png"))
display(Image(output_dir / "sur_raman_aniso.png"))
display(Image(output_dir / "sur_raman_aniso_low.png"))

计算和绘制水分子（H₂O）与羟基（OH）的和频产生（SFG）光谱。加载两者的坐标、晶胞、原子偶极矩及极化率数据并合并，通过空间权重函数处理特定区域的分子，计算基于极化率对角分量与偶极矩 z 分量的分子内和分子间相关函数，经傅里叶变换得到频域数据后，分别生成 H₂O 和 OH 在 2800-4000 cm⁻¹ 波数范围内的 SFG 光谱图，展示二阶非线性极化率虚部与波数的关系并保存为图像文件。

In [ ]:
dt = 0.0005
window = 50000

h2o = np.load(sample_dir / "h2o.npy")
atomic_dipole = np.load(sample_dir / "atomic_dipole_wan.npy").reshape(h2o.shape[0], -1, 3)
atomic_polar = np.load(sample_dir / "atomic_polar_wan.npy").reshape(h2o.shape[0], -1, 3, 3)
name = "SFG.dat"

md_spectra.compute_surface_sfg_h2o(
    h2o=h2o,
    atomic_dipole=atomic_dipole,
    atomic_polar=atomic_polar,
    dt=dt,
    window=window,
    z0=22.5,
    zc=2.5,
    zw=2.6,
    rc=6.75,
    width=50,
    temperature=330.0,
    save_plot=output_dir / "sfg.png",
    save_data=output_dir / name,
)

display(Image(output_dir / "sfg.png"))